<a href="https://colab.research.google.com/github/v-puppala/Palette_Inpainting_CelebaHQ_256_ipynb_for_Staining_Data/blob/main/Palette_Inpainting_CelebaHQ_256_ipynb_for_Staining_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Image inpainting
A demo for image inpainting on Celeba-HQ based on [Palette](https://github.com/Janspiry/Palette-Image-to-Image-Diffusion-Models).

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil

source_file = '/content/drive/MyDrive/PytorchModels/10.state'
source_file1='/content/drive/MyDrive/PytorchModels/10_Network.pth'
destination_folder = '/content/Palette-Image-to-Image-Diffusion-Models'

shutil.move(source_file, destination_folder)
shutil.move(source_file1, destination_folder)


Error: ignored

## Setup

In [2]:
%cd /content/
!git clone https://github.com/v-puppala/Palette-Image-to-Image-Diffusion-Models

/content
Cloning into 'Palette-Image-to-Image-Diffusion-Models'...
remote: Enumerating objects: 317, done.
remote: Counting objects: 100% (146/146), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 317 (delta 115), reused 101 (delta 101), pack-reused 171
Receiving objects: 100% (317/317), 6.87 MiB | 17.98 MiB/s, done.
Resolving deltas: 100% (176/176), done.


In [ ]:
%cd /content/Palette-Image-to-Image-Diffusion-Models/


/content/Palette-Image-to-Image-Diffusion-Models


### Upload data

In [ ]:
%cd /content/Palette-Image-to-Image-Diffusion-Models/
!mkdir -p input
%cd /content/Palette-Image-to-Image-Diffusion-Models/input/


/content/Palette-Image-to-Image-Diffusion-Models
/content/Palette-Image-to-Image-Diffusion-Models/input


### Get the GPU

*   Turn on hardware acceleration under `Runtime -> Change Runtime Type -> Hardware accelerator -> GPU`
*   Use this command to ensure that the connected machine has a GPU:

In [ ]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

/bin/bash: line 1: nvidia-smi: command not found


### Clone repo

### Install requirements

In [ ]:
!pip list
# The default environment can working properly

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.8.6
aiosignal                        1.3.1
alabaster                        0.7.13
albumentations                   1.3.1
altair                           4.2.2
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.1.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.13.1
backcall                         0.2.0
beautifulsoup4                   4.11.2
bidict                           0.22.1
b

**Shrinking Dataset**

In [ ]:
# import os
# import random
# import shutil

# mask_files=os.listdir('/content/drive/MyDrive/output/mask')
# sample_size=int(.2*len(mask_files))

# new_mask_path='/content/drive/MyDrive/output/new_mask1'
# new_files_mask=random.sample(mask_files,sample_size)

# if not os.path.exists(new_mask_path):
#     os.makedirs(new_mask_path)

# for file in new_files_mask:
#     shutil.copy(os.path.join('/content/drive/MyDrive/output/mask', file), new_mask_path)



# im_files=os.listdir('/content/drive/MyDrive/output/im')

# new_im_path='/content/drive/MyDrive/output/new_im1'
# new_files_im=random.sample(im_files,sample_size)

# if not os.path.exists(new_im_path):
#     os.makedirs(new_im_path)

# for file in new_files_im:
#     shutil.copy(os.path.join('/content/drive/MyDrive/output/im', file), new_im_path)






In [3]:
import json

# Define a dictionary
data={
    "name": "inpainting_celebahq",
    "gpu_ids": [0],
    "seed" : -1,
    "finetune_norm": False,

    "path": {
        "base_dir": "experiments",
        "code": "code",
        "tb_logger": "tb_logger",
        "results": "results",
        "checkpoint": "checkpoint",
        "resume_state": "200"

          },

    "datasets": {
        "train": {
            "which_dataset": {
                "name": "InpaintDataset",
                "args":{
                    "data_root": "/content/drive/MyDrive/output/new_im1",
                    "data_len": -1,
                    "mask_config": {
                        "mask_mode": "hybrid"
                    }
                }
            },
            "dataloader":{
                "validation_split": 2,
                "args":{
                    "batch_size": 3,
                    "num_workers": 4,
                    "shuffle": True,
                    "pin_memory": True,
                    "drop_last": True
                },
                "val_args":{
                    "batch_size": 1,
                    "num_workers": 4,
                    "shuffle": False,
                    "pin_memory": True,
                    "drop_last": False
                }
            }
        },
        "test": {
            "which_dataset": {
                "name": "InpaintDataset",
                "args":{
                    "data_root": "/content/drive/MyDrive/output/new_mask1",
                    "mask_config": {
                        "mask_mode": "center"
                    }
                }
            },
            "dataloader":{
                "args":{
                    "batch_size": 1,
                    "num_workers": 1,
                    "pin_memory": True
                }
            }
        }
    },

    "model": {
        "which_model": {
            "name": ["models.model", "Palette"],
            "args": {
                "sample_num": 8,
                "task": "inpainting",
                "ema_scheduler": {
                    "ema_start": 1,
                    "ema_iter": 1,
                    "ema_decay": 0.9999
                },
                "optimizers": [
                    { "lr": 5e-5, "weight_decay": 0}
                ]
            }
        },
        "which_networks": [
            {
                "name": ["models.network", "Network"],
                "args": {
                    "init_type": "kaiming",
                    "module_name": "guided_diffusion",
                    "unet": {
                        "in_channel": 6,
                        "out_channel": 3,
                        "inner_channel": 64,
                        "channel_mults": [
                            1,
                            2,
                            4,
                            8
                        ],
                        "attn_res": [
                            16
                        ],
                        "num_head_channels": 32,
                        "res_blocks": 2,
                        "dropout": 0.2,
                        "image_size": 256
                    },
                    "beta_schedule": {
                        "train": {
                            "schedule": "linear",
                            "n_timestep": 2000,
                            "linear_start": 1e-6,
                            "linear_end": 0.01
                        },
                        "test": {
                            "schedule": "linear",
                            "n_timestep": 1000,
                            "linear_start": 1e-4,
                            "linear_end": 0.09
                        }
                    }
                }
            }
        ],
        "which_losses": [
            "mse_loss"
        ],
        "which_metrics": [
            "mae"
        ]
    },

    "train": {
        "n_epoch": 1e8,
        "n_iter": 1e8,
        "val_epoch": 5,
        "save_checkpoint_epoch": 10,
        "log_iter": 1e3,
        "tensorboard" : True
    },

    "debug": {
        "val_epoch": 1,
        "save_checkpoint_epoch": 1,
        "log_iter": 2,
        "debug_split": 50
    }
}

# Define the file path
file_path = '/content/sample.json'

# Write the dictionary to a JSON file
with open(file_path, 'w') as f:
    json.dump(data, f)

print("JSON file created successfully.")


JSON file created successfully.


In [ ]:
import json

data1={
    "name": "new_mask1",
    "gpu_ids": [0],
    "seed" : -1,
    "finetune_norm": False,
    "path": {
        "base_dir": "experiments",
        "code": "code",
        "tb_logger": "tb_logger",
        "results": "results",
        "checkpoint": "checkpoint",
        "resume_state": "/content/Palette-Image-to-Image-Diffusion-Models/10"
    },

    "datasets": {
        "train": {
            "which_dataset": {
                "name": "new_im1",
                "args":{
                    "data_root": "/content/drive/MyDrive/output/new_im1",
                    "data_len": -1,
                    "mask_config": {
                        "mask_mode": "hybrid"
                    }
                }
            },
            "dataloader":{
                "validation_split": 2,
                "args":{
                    "batch_size": 3,
                    "num_workers": 4,
                    "shuffle": True,
                    "pin_memory": True,
                    "drop_last": True
                },
                "val_args":{
                    "batch_size": 1,
                    "num_workers": 4,
                    "shuffle": False,
                    "pin_memory": True,
                    "drop_last": False
                }
            }
        },
        "test": {
            "which_dataset": {
                "name": "new_mask1",
                "args":{
                    "data_root": "/content/drive/MyDrive/output/new_mask1",
                    "mask_config": {
                        "mask_mode": "center"
                    }
                }
            },
            "dataloader":{
                "args":{
                    "batch_size": 8,
                    "num_workers": 4,
                    "pin_memory": True
                }
            }
        }
    },

    "model": {
        "which_model": {
            "name": ["models.model", "Palette"],
            "args": {
                "sample_num": 8,
                "task": "inpainting",
                "ema_scheduler": {
                    "ema_start": 1,
                    "ema_iter": 1,
                    "ema_decay": 0.9999
                },
                "optimizers": [
                    { "lr": 5e-5, "weight_decay": 0}
                ]
            }
        },
        "which_networks": [
            {
                "name": ["models.network", "Network"],
                "args": {
                    "init_type": "kaiming",
                    "module_name": "guided_diffusion",
                    "unet": {
                        "in_channel": 6,
                        "out_channel": 3,
                        "inner_channel": 64,
                        "channel_mults": [
                            1,
                            2,
                            4,
                            8
                        ],
                        "attn_res": [
                            16
                        ],
                        "num_head_channels": 32,
                        "res_blocks": 2,
                        "dropout": 0.2,
                        "image_size": 256
                    },
                    "beta_schedule": {
                        "train": {
                            "schedule": "linear",
                            "n_timestep": 2000,
                            "linear_start": 1e-6,
                            "linear_end": 0.01
                        },
                        "test": {
                            "schedule": "linear",
                            "n_timestep": 1000,
                            "linear_start": 1e-4,
                            "linear_end": 0.09
                        }
                    }
                }
            }
        ],
        "which_losses": [
            "mse_loss"
        ],
        "which_metrics": [
            "mae"
        ]
    },

    "train": {
        "n_epoch": 1e8,
        "n_iter": 1e8,
        "val_epoch": 5,
        "save_checkpoint_epoch": 10,
        "log_iter": 1e3,
        "tensorboard" : True
    },

    "debug": {
        "val_epoch": 1,
        "save_checkpoint_epoch": 1,
        "log_iter": 2,
        "debug_split": 50
    }
    }

# Define the file path
file_path = '/content/Palette-Image-to-Image-Diffusion-Models/config/inference.json'

# Write the dictionary to a JSON file
with open(file_path, 'w') as f:
    json.dump(data1, f)

print("JSON file created successfully.")


JSON file created successfully.


### Prepare pertrained-model
Two ways to prepare your model are provided below.



#### Download from the google driver
1. Go [here](https://drive.google.com/drive/folders/13YZ2UAmGJ-b7DICr-FDAPM7gctreJEoH?usp=sharing), right click on `.pth` file, and hit `Make a copy`. Next, locate the files in your own google drive, and rename them from `Copy of 200_Network.pth` to `200_Network.pth`. Note, `.state` file is not required for the simple inference.
2. Next, right click on each file, and hit `Get link`. Click on `Restricted`, and change it to `Anyone with the link...`, as `Viewer`. Now, copy the links. They should look something like this: `https://drive.google.com/file/d/<file id>/view?usp=sharing`. Copy the file id, and paste them into the `gdown` commands below.


In [ ]:
%cd /content/Palette-Image-to-Image-Diffusion-Models/
!gdown --id 1gzNs8_9LuysjbWsHftXdHp0jDYOyIgY1

/content/Palette-Image-to-Image-Diffusion-Models
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1gzNs8_9LuysjbWsHftXdHp0jDYOyIgY1
To: /content/Palette-Image-to-Image-Diffusion-Models/200_Network.pth
100% 251M/251M [00:03<00:00, 65.1MB/s]


#### Upload from the localhost
If `gdown` command did not work, you can manually upload files for the pre-trained model.
1. Download the [files](https://drive.google.com/drive/folders/13YZ2UAmGJ-b7DICr-FDAPM7gctreJEoH?usp=sharing) to localhost.
2. Click the `files` button on the left hand side of the screen, and find the folder labled `Palette-Image-to-Image-Diffusion-Models`. Click the three dots on the right of the folder, and click `upload`. Then, select the model files that you downloaded.

### Edit configure file

In [ ]:
%cd /content/Palette-Image-to-Image-Diffusion-Models/config/

/content/Palette-Image-to-Image-Diffusion-Models/config


#### Create patchfile

Create patchfile, it will update the configuration file to inference for centering mask.

#### Apply the patch file
The error `patch unexpectedly ends in middle of line` can be ignored.

In [ ]:
!apt-get install dos2unix
!dos2unix inpainting_celebahq.json
!patch < inpainting_celebahq.patch

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  dos2unix
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 384 kB of archives.
After this operation, 1,367 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 dos2unix amd64 7.4.2-2 [384 kB]
Fetched 384 kB in 2s (237 kB/s)
Selecting previously unselected package dos2unix.
(Reading database ... 120874 files and directories currently installed.)
Preparing to unpack .../dos2unix_7.4.2-2_amd64.deb ...
Unpacking dos2unix (7.4.2-2) ...
Setting up dos2unix (7.4.2-2) ...
Processing triggers for man-db (2.10.2-1) ...
dos2unix: converting file inpainting_celebahq.json to Unix format...
patching file inpainting_celebahq.json


In [ ]:
import torch
from torch.utils.data import Dataset

class CD8Dataset(Dataset):
    def __init__(self, data_path, transform=None):
        # Initialize data, file paths, or any other necessary components
        self.data = [
    {'image_path': 'path_to_image_1', 'label': 0},
    {'image_path': 'path_to_image_2', 'label': 1},
    # Add more data samples as needed
]

        self.transform = transform

    def __len__(self):
        # Return the total number of samples in the dataset
        return len(self.data)

    def __getitem__(self, idx):
        # Retrieve an individual item from the dataset at the specified index
        sample = self.data[idx]

        # Apply any necessary transformations to the sample
        if self.transform:
            sample = self.transform(sample)

        return sample

# Example usage
data_path = "/content/drive/MyDrive/output/new_mask1"
transform = None  # Add your custom transformations if required
cd8_dataset = CD8Dataset(data_path, transform)

# Use DataLoader to create an iterable dataset
dataloader = torch.utils.data.DataLoader(cd8_dataset, batch_size=4, shuffle=True, num_workers=2)


TypeError: ignored

## Inference
It will cost several minutes depending on allocated GPU.

Output images will be located under ./experiments/\<some folder\>/results/test.

In [ ]:
%cd /content/Palette-Image-to-Image-Diffusion-Models/

/content/Palette-Image-to-Image-Diffusion-Models


In [ ]:
%rm '/content/drive/MyDrive/output/new_mask1/YTMA79-10_12_1437_184.png_r438_c181_rot2.png'

rm: cannot remove '/content/drive/MyDrive/output/new_mask1/YTMA79-10_12_1437_184.png_r438_c181_rot2.png': No such file or directory


In [ ]:
#init_obj is in parser.py
#create_model is called in run.py and defined in models init.py

In [ ]:
%cd /content/Palette-Image-to-Image-Diffusion-Models/
!python run.py -p train -c /content/sample.json


/content/Palette-Image-to-Image-Diffusion-Models
export CUDA_VISIBLE_DEVICES=0
/content/Palette-Image-to-Image-Diffusion-Models/run.py:28: UserWarning: You have chosen to use cudnn for accleration. torch.backends.cudnn.enabled=True
  warnings.warn('You have chosen to use cudnn for accleration. torch.backends.cudnn.enabled=True')
2023-12-06 20:59:40.079475: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-06 20:59:40.079528: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-06 20:59:40.079561: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-06 20:59:41.201253: W t

In [ ]:
%cd /content/Palette-Image-to-Image-Diffusion-Models/
!python run.py -c config/inference.json -p test

/content/Palette-Image-to-Image-Diffusion-Models
export CUDA_VISIBLE_DEVICES=0
/content/Palette-Image-to-Image-Diffusion-Models/run.py:28: UserWarning: You have chosen to use cudnn for accleration. torch.backends.cudnn.enabled=True
  warnings.warn('You have chosen to use cudnn for accleration. torch.backends.cudnn.enabled=True')
2023-12-01 20:30:36.156306: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-01 20:30:36.156357: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-01 20:30:36.156384: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-01 20:30:37.269446: W t

In [ ]:

|||}shutil.move('/content/Palette-Image-to-Image-Diffusion-Models/experiments','/Users/vishalpuppala/Python')


Error: ignored

In [ ]:
!cp <output-file-name> /content/drive/MyDrive/<destination-folder-name>


In [ ]:
import torch
import torchvision.models as models


loaded_model = torch.load('/content/drive/MyDrive/PytorchModels/10_Network.pth')


In [ ]:
%cd /content/Palette-Image-to-Image-Diffusion-Models/

/content/Palette-Image-to-Image-Diffusion-Models


In [ ]:
from PIL import Image
from torchvision import transforms
import os

# Define image preprocessing transforms
preprocess = transforms.Compose([
    transforms.ToTensor()
])
mask_config={"mask_mode": "hybrid"}
#model1=new_im1(data_root=data_root,mask_config=mask_config)
image_dir='/content/drive/MyDrive/output/new_mask1'
for image_path in os.listdir(image_dir):
    image = Image.open(os.path.join(image_dir, image_path))
    image = preprocess(image)
    image = image.unsqueeze(0)

    outputs = new_im1(data_root=image_path,mask_config=mask_config)

    # Interpret the model's output (e.g., for classification)
    _, predicted_class = outputs.max(1)


AssertionError: ignored

In [ ]:
import importlib
import inspect
import models  # Replace 'my_package' with the actual package name

# Load the package to ensure it's imported
importlib.import_module('models')

# Get a list of all modules in the package
package_modules = [name for name, _ in inspect.getmembers(models, inspect.ismodule)]
package_modules

[]

In [ ]:
import torch
import torchvision.models as models

#https://machinelearningmastery.com/save-and-load-your-pytorch-m
#https://pytorch.org/tutorials/beginner/saving_loading_models.html

In [ ]:
loaded_model()


AttributeError: ignored

In [ ]:
}}\

/]\\=]

In [ ]:

}=\]
]]
\

In [ ]:
|\

In [ ]:
\

\\
\-=p--p[-===]-==]\==\]==\]\]=\